<a href="https://colab.research.google.com/github/bhavinithak/DeepLearning_Pytorch/blob/main/US_Patent_phrase_to_phrase_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use GPU

In [ ]:
!pip install kaggle

In [ ]:
!pip install datasets #runtime > restart session after pip install before importing

In [ ]:
!pip install transformers[torch] #runtime > restart session after pip install before importing

#Reading data from Kaggle

In [7]:
creds='{"username":"","key":""}'

In [8]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [9]:
path = Path('us-patent-phrase-to-phrase-matching')

In [10]:
if not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [11]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [12]:
import pandas as pd

In [13]:
df= pd.read_csv(path/'train.csv')

In [14]:
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [15]:
df.shape

(36473, 5)

In [16]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [17]:
df['input']='TEXT1: '+df.context + '; TEXT2: ' + df.target + '; ANC1: '+df.anchor

In [18]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

#Tokenization

In [19]:
from datasets import Dataset, DatasetDict #runtime > restart session after pip install before importing

In [20]:
ds=Dataset.from_pandas(df)

In [21]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [22]:
model_nm='microsoft/deberta-v3-small'

In [23]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz=AutoTokenizer.from_pretrained(model_nm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [24]:
tokz.tokenize("G'day folks. I am Bhavini")

['▁G', "'", 'day', '▁folks', '.', '▁I', '▁am', '▁Bhav', 'ini']

In [25]:
def tok_func(x):
  return tokz(x['input'])

In [26]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [27]:
row=tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [28]:
tokz.vocab["▁of"] ## this is not underscore(_) it is special token taken from o/p of tokz.tokenize("G'day folks. I am Bhavini")

265

In [29]:
tok_ds=tok_ds.rename_columns({'score':'labels'})

#Test and validation sets

In [30]:
eval_df = pd.read_csv(path/'test.csv') #test set
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [39]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [31]:
dds=tok_ds.train_test_split(0.25, seed=42)
dds # train and validation set

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

#Metrics

In [32]:
import numpy as np

In [33]:
def corr(x,y):
  return np.corrcoef(x,y)[0][1]

In [34]:
def corr_d(eval_pred):
  return {'pearson':corr(*eval_pred)}

#Training

In [35]:
from transformers import TrainingArguments, Trainer

In [36]:
bs=128
epochs=4

In [37]:
lr=8e-5 #learning rate

In [38]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [40]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.027960,0.796811
2,No log,0.021929,0.823581
3,0.038000,0.022062,0.833889
4,0.038000,0.023100,0.834454


TrainOutput(global_step=856, training_loss=0.027762085477882458, metrics={'train_runtime': 221.4922, 'train_samples_per_second': 493.995, 'train_steps_per_second': 3.865, 'total_flos': 717218678299260.0, 'train_loss': 0.027762085477882458, 'epoch': 4.0})

#Prediction

In [43]:
np.set_printoptions(precision=2, suppress=True)

In [44]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.52],
       [ 0.67],
       [ 0.49],
       [ 0.38],
       [-0.02],
       [ 0.57],
       [ 0.53],
       [-0.02],
       [ 0.33],
       [ 1.1 ],
       [ 0.28],
       [ 0.24],
       [ 0.76],
       [ 1.01],
       [ 0.76],
       [ 0.44],
       [ 0.33],
       [-0.03],
       [ 0.64],
       [ 0.38],
       [ 0.52],
       [ 0.27],
       [ 0.18],
       [ 0.25],
       [ 0.58],
       [-0.01],
       [-0.03],
       [ 0.  ],
       [-0.02],
       [ 0.72],
       [ 0.34],
       [ 0.04],
       [ 0.72],
       [ 0.55],
       [ 0.46],
       [ 0.22]])

In [45]:
preds=np.clip(preds,0,1)
preds

array([[0.52],
       [0.67],
       [0.49],
       [0.38],
       [0.  ],
       [0.57],
       [0.53],
       [0.  ],
       [0.33],
       [1.  ],
       [0.28],
       [0.24],
       [0.76],
       [1.  ],
       [0.76],
       [0.44],
       [0.33],
       [0.  ],
       [0.64],
       [0.38],
       [0.52],
       [0.27],
       [0.18],
       [0.25],
       [0.58],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.72],
       [0.34],
       [0.04],
       [0.72],
       [0.55],
       [0.46],
       [0.22]])